# **Extrator de sumários das Atas do DAR**

Os códigos montados nesse *notebook* tem como objetivo extrair os sumários que estão nas Atas das Reuniões Plenárias que estão dentro do Diário da Assembléia da República, vale lembrar que essas atas já foram extraídas em formato de arquivo de texto simples, com codificação UTF-8 [deste site](https://debates.parlamento.pt/catalogo/r3/dar/01/14/02).

Para isso tentaremos fazer um pequeno *pipeline* de cada uma das ações e métodos que deveremos implementar para poder extrair fielmente o sumário de qualquer Ata de forma geral, assim sempre que tivermos novas Atas é só rodar esse código para termos um sumário parcialmente limpo.

*   Abrir as pastas e documentos onde estão os arquivos
*   Analisar principais padrões de onde estão os sumários
*   Identificar padrões de troca de página
*   Identificar padrões de troca de parágrafo
*   Extrair os sumários
*   Aplicar limpeza nos arquivos

# **Abrindo local dos documentos:**

Vamos abrir então nossa pasta do Drive que contém já nossas Atas em formato de arquivo de texto e já extrair uma lista com esses documentos.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=1)

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Iniciação Científica/AI Democracy/Atas em .txt'

/content/drive/MyDrive/Iniciação Científica/AI Democracy/Atas em .txt


In [3]:
import os 

#Listando documentos
document_names = os.listdir()

#Abrindo os documentos e extraindo os textos
list_texts = []
for document in document_names:
    with open(document, 'r') as doc:
        text = doc.read()
        list_texts.append(text)

#Imprimindo uma parte do texto da primeira ata
print(list_texts[0][:3000])

Quinta-feira, 17 de setembro de 2020 I Série — Número 1



XIV LEGISLATURA 2.ª SESSÃO LEGISLATIVA (2020-2021) 

 REUNIÃOPLENÁRIADE16DESETEMBRODE 2020 

Presidente: Ex.mo Sr. Eduardo Luís Barreto Ferro Rodrigues 

Secretários: Ex.mos Srs. Maria da Luz Gameiro Beja Ferreira Rosinha Duarte Rogério Matos Ventura Pacheco 





S U M Á R I O 

O Presidente declarou aberta a sessão às 15 horas e 2 

minutos. Deu-se conta da entrada na Mesa dos Projetos de Lei n.os 

485 a 505/XIV/1.ª, dos Projetos de Resolução n.os 612 a 617/XIV/1.ª, 619 a 624/XIV/1.ª e 626/XIV/1.ª e do Inquérito Parlamentar n.º 6/XIV/1.ª. 

Em declaração política, o Deputado Luís Moreira Testa (PS) salientou a importância de serem levadas a cabo políticas de coesão territorial para combater as assimetrias do 

País, nomeadamente através da valorização do interior e do estabelecimento de relações transfronteiriças entre Portugal e Espanha. No fim, respondeu a pedidos de esclarecimento dos Deputados José Luís Ferreira (PEV), A

# **Padrões de localização do sumário:**

Analisando alguns dos sumários podemos perceber rapidamente que eles se encontram a partir da `string` "S U M Á R I O" e vai até uma linha que contém as palavras "presidente", "encerrou", "sessão".

Então podemos pensar em tokenizar nosso texto em cada quebra de linha, excluir tudo até "S U M Á R I O" e tudo depois da linha que contém "presidente", "encerrou", "sessão".

In [4]:
def extract_summary(list_texts):

    texts = list_texts.copy()
    summaries = []

    for i in range(len(texts)):

        #Definindo onde começa o sumário e limpando antes disso
        text = texts[i]
        text = text[text.find('S U M Á R I O'):]

        #Trabalhar com as linhas é mais fácil
        text = text.split(sep='\n')

        #Aplicando o método pensado
        for j in range(len(text)):

            #Definição dos termos importantes
            presidente = 'Presidente' in text[j]
            encerrou = 'encerrou' in text[j]
            sessão = 'sessão' in text[j]
            ponto = '.' in text[j]
            
            #Checando se todos os termos aparecem na linha
            if (presidente and encerrou and sessão and ponto):
                text = text[:j+1]
                text = '\n'.join(text)
                break

            #Se não tiver o ponto aconteceu algum erro de formatação
            #Vamos então buscar onde termina o parágrafo
            elif (presidente and encerrou and sessão and not ponto):
                for k in range(1, len(text[j+1:]) + 1):
                    if '.' in text[j+k] and 'horas' in text[j+k]:
                        text = text[:j+k+1]
                        text = '\n'.join(text)
                        break
                break

        #Como sempre terminamos com um .join()
        #Se o texto ainda for uma lista aconteceu um erro
        if type(text) is list:
            print("Erro na Ata de index {}".format(i))

        summaries.append(text)
    return summaries

summaries = extract_summary(list_texts)

Erro na Ata de index 32


Beleza, nosso extrator de sumários funcionou muito bem. Olhando manualmente o erro na Ata 33 (index 32), pode-se perceber que também é marcado no sumário quando o Presidente encerrou a sessão, mas por ser véspera de Natal ele faz algumas outras considerações com a finalização.

De qualquer forma vamos arrumar isso rapidamente.

In [5]:
for i in range(len(summaries[32])):
    if 'Encerrou, depois, a sessão eram 19 horas e 27 minutos.' in summaries[32][i]:
        summaries[32] = summaries[32][:i + 1]
        summaries[32] = '\n'.join(summaries[32])
        break

Vamos printar o último token da texto só por desencargo de consciência, se algum estiver muito diferente vai saltar aos olhos facilmente.

In [6]:
i = 0
for text in summaries:
    i += 1
    text = text.split('\n')
    print(i, '-', text[-1])

1 - O Presidente (Fernando Negrão) encerrou a sessão eram 18 horas e 59 minutos. 
2 - O Presidente (Fernando Negrão) encerrou a sessão eram 19 horas e 3 minutos. 
3 - O Presidente (Fernando Negrão) encerrou a sessão eram 13 horas e 55 minutos. 
4 - O Presidente encerrou a sessão eram 18 horas e 28 minutos. 
5 - O Presidente encerrou a sessão eram 18 horas e 13 minutos. 
6 - O Presidente (Fernando Negrão) encerrou a sessão eram 14 horas. 
7 - O Presidente (Fernando Negrão) encerrou a sessão eram 18 horas e 56 minutos. 
8 - O Presidente (José Manuel Pureza) encerrou a sessão eram 18 horas e 11 minutos. 
9 - O Presidente (José Manuel Pureza) encerrou a sessão eram 13 horas e 32 minutos. 
10 - O Presidente (José Manuel Pureza) encerrou a sessão eram 19 horas e 12 minutos. 
11 - O Presidente (José Manuel Pureza) encerrou a sessão eram 18 horas e 29 minutos. 
12 - O Presidente encerrou a sessão eram 12 horas e 33 minutos. 
13 - O Presidente (José Manuel Pureza) encerrou a sessão eram 20 hora

Bom basicamente está OK, todos terminam com o presidente declarando o final da sessão, claro que tem algumas que saltam aos olhos por serem diferentes, mas é questão de paragrafação. Provavelmente os parágrafos que estão maiores tiveram algum problema durante as quebras de linha.

# **Padrões de troca de página:**

Vamos agora pensar nos principais padrões de troca de página que existem.

A maioria das atas estão entre a primeira e segunda páginas, fazendo com que extrair essa primeira divisória seja bem fácil onde se encontra a `string` "I SÉRIE — NÚMERO X" em que o número representa qual o número da própria ata, seguido do número 2 da página. Uma observação sobre isso é que precisamos identificar quando ocorre também troca de parágrafo entre as páginas ou não.

Outra troca são para atas que contém 3 páginas, essas atas contém a data, uma coisa que podemos fazer é checar o mês que está em caixa alta e a partir dele remover dia, ano e a marcação da terceira página.

Vale notar que é alternado, ou seja, páginas pares estão com SÉRIE e páginas ímpares com as DATAS, isso é útil até para se for feita uma limpeza mais geral dos documentos.

Observe: Testando deu para perceber que a série sempre está em um index par, mas não vou usar esse fato

In [7]:
def find_dates(list_texts):
    dates = []
    for text in list_texts:
        text = text.split('\n')
        while len(text[0]) < 2:
            text.pop(0)
        text = text[0]
        text = text[text.find(', ') + 2:text.find(' I')].upper()
        dates.append(text)
    return dates


def remove_page_marks(list_texts, list_dates):
    #A serie pode aparecer em dois formatos
    #Como vamos testar se existem também quebra de parágrafo
    #Teremos 4 possíveis formatos
    
    texts = list_texts.copy()
    dates = list_dates.copy()

    for i in range(len(texts)):

        #Primeiro vamos retirar o cabeçalho de SERIE
        serie_1 = '.  \n\n \n\nI SÉRIE — NÚMERO {} \n\n\n\n2 \n\n'.format(i + 1)
        serie_2 = '.  \n\n \n\nI SÉRIE — NÚMERO {} \n\n\n\n\n\n2 \n\n'.format(i + 1)
        serie_3 = '  \n\n \n\nI SÉRIE — NÚMERO {} \n\n\n\n2 \n\n'.format(i + 1)
        serie_4 = '  \n\n \n\nI SÉRIE — NÚMERO {} \n\n\n\n\n\n2 \n\n'.format(i + 1)

        if serie_1 in texts[i]:
            texts[i] = texts[i].replace(serie_1, '\n\n')
        elif serie_2 in texts[i]:
            texts[i] = texts[i].replace(serie_2, '\n\n')
        elif serie_3 in texts[i]:
            texts[i] = texts[i].replace(serie_3, ' ')
        elif serie_4 in texts[i]:
            texts[i] = texts[i].replace(serie_4, ' ')
        else:
            print("Erro na Ata de index {}".format(i))
        
        #Agora vamos fazer o mesmo processo para as datas
        date_1 = '.  \n\n \n\n{} \n\n3 \n\n'.format(dates[i])
        date_2 = '  \n\n \n\n{} \n\n3 \n\n'.format(dates[i])

        if date_1 in texts[i]:
            texts[i] = texts[i].replace(date_1, '\n\n')
        elif date_2 in texts[i]:
            texts[i] = texts[i].replace(date_2, ' ')
        #Aqui não podemos colocar um else
        #Pelo fato que todas as que não tem 3 pgs iam cair aqui

    return texts

dates = find_dates(list_texts)
clean_texts = remove_page_marks(summaries, dates)

Erro na Ata de index 37


Por algum motivo esqueceram de colocar o cabeçalho na Ata 38 (index 37) vamos então arrumar essa formatação errada que aconteceu

In [8]:
clean_texts[37] = clean_texts[37].replace('  \n\n \n\n\n\n\n\n', ' ')

# **Padrões de troca de parágrafos:**

Bom, agora já tiramos as imperfeições que existem no nosso texto basta apenas deixar corretamente um parágrafo por linha, um jeito bom de fazer isso é separar nosso texto por quebra de linha e ver se aquela linha termina em ponto final.

In [75]:
def clean_tokens(tokenized_text):

    text = tokenized_text.copy()
    to_del = []

    for i in range(len(text)):
        if 'S U M Á R I O' in text[i]:
            text[i] = text[i].replace('S U M Á R I O', '')
        elif len(text[i]) <= 2:        
            to_del.append(i)

    for delete in to_del:
        text.pop(delete)
    
    return text
    

def clean_paragraphs(list_texts):

    texts = list_texts.copy()
    new_texts = []

    for text in texts:
        text = text.replace('. \n\n', '.\n')
        text = text.replace(' \n\n', '')
        text = text.split('\n')
        text = clean_tokens(text)
        text = clean_tokens(text)
        text = clean_tokens(text)
        compound = ''
        for line in text:
            if '.' in line[-1]:
                compound = compound + '\n'  + line
            else:
                compound = compound + ' '  + line
        new_texts.append(compound[1:])

    return new_texts

total_clean = clean_paragraphs(clean_texts)
print(total_clean[1])

O Presidente declarou aberta a sessão às 15 horas e 4minutos. Deu-se conta da entrada na Mesa das Propostas deResolução n.os 11 e 12/XIV/2.ª, dos Projetos de Lei n.os 506, 507 e 509 a 511/XIV/2.ª e dos Projetos de Resolução n.os 627 a 631/XIV/2.ª.
Foi discutido o Projeto de Resolução n.º 561/XIV/1.ª (PS) — Recomenda ao Governo que crie uma tarifa social de acesso a serviços de internet. Intervieram os Deputados Filipe Pacheco (PS), Inês de Sousa Real (PAN), João Cotrim deFigueiredo (IL), Hugo Martins de Carvalho (PSD), Mariana Silva (PEV), Bruno Dias (PCP), Isabel Pires (BE), João Gonçalves Pereira (CDS-PP) e André Ventura (CH). Foi discutido, na generalidade, o Projeto de Lei n.º 474/XIV/1.ª (PSD) — Programa especial de apoio social aos ex-trabalhadores da COFACO. Na abertura do debate, pronunciou-se o Deputado Paulo Moniz (PSD), seguindo-se no uso da palavra os Deputados João Azevedo Castro (PS), João Pinho de Almeida (CDS-PP), Isabel Pires (BE) e Alma Rivera (PCP)
Foram apreciados, 

# Salvando os textos no Drive:

Agora que finalizamos nossa limpeza temos que mandar os textos de volta para o Drive, temos já uma pasta vamos apenas jogar os textos lá.

In [44]:
cd ../Sumários em .txt

/content/drive/My Drive/Iniciação Científica/AI Democracy/Sumários em .txt


In [77]:
i = 0
for text in total_clean:
    i += 1
    name = "Sumário - "
    if i < 10:
        name = name + '00' + str(i) + '.txt'
    else:
        name = name + '0' + str(i) + '.txt'
    with open(name, 'w') as f:
        f.write(text)